In [1]:
import pandas as pd
import numpy as np

# features - model
model_features_list = ['transcript', 'position', 'nucleotides', 'gene_id','dwelling_time', 'std', 'mean', 'label']

# features - preprocessing
key_columns = ['label'] # columns that cannot be null
non_nan_cols = []

# features - encoding
target_encode_cols = ['nucleotides']
target_col = 'label'
numeric_cols = ['dwelling_time', 'std', 'mean']

# oversampling / undersampling
undersampling_strategy = 0.15
oversampling_strategy = 0.20

# data splitting
stratify_col = 'label' # column used to stratify data
seed = 42 # random state
id_col = ['gene_id']
position_col = ['transcript', 'position']
    # the following 3 percentages should sum to 1
train_percent = 0.8 # percentage of train data
validation_percent = 0.1 # percentage of validation data
test_percent = 0.1 # percentage of test data

In [4]:
def get_percent(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [5]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from category_encoders import TargetEncoder


class Preprocessing:
    def __init__(self, df):
        self.df = df

        self.columns_to_map = None
        self.df_train = None
        self.df_test = None
        self.df_val = None
        self.X_train = None
        self.y_train = None
        self.X_val = None
        self.y_val = None
        self.X_test = None
        self.y_test = None
        self.pipe = None
        
    def model_features_and_clean(self):
        '''Function to select features for modelling and clean the initial raw data. 
            Removes rows which contain NAN values in key_columns, and converts all NAN values to 0 for non_nan_cols
        :Parameters:
        ------------
            self.df: DataFrame
                DataFrame containing raw data
        :Returns:
        ---------
            self.df: DataFrame
                DataFrame containing features required for modelling
        '''
        self.df = self.df[model_features_list]
        self.df[non_nan_cols] = self.df[non_nan_cols].fillna(0)
        self.df = self.df.dropna(subset=key_columns)
        return self.df
    
    def feature_eng(self):
        self.df = pd.DataFrame(self.df.groupby(['gene_id', 'transcript', 'position', 'nucleotides', 'label'], as_index=False)
                            .agg({'dwelling_time': [get_percent(25), get_percent(50), get_percent(75), np.mean], 
                                    'std': [get_percent(25), get_percent(50), get_percent(75), np.mean], 
                                    'mean': [get_percent(25), get_percent(50), get_percent(75), np.mean]}))
        self.df.columns = ['gene_id', 'transcript', 'position', 'nucleotides', 'label',
                            'dwelling_time_25', 'dwelling_time_50', 'dwelling_time_75', 'dwelling_time_mean', 
                            'std_25', 'std_50', 'std_75', 'std_mean', 
                            'mean_25', 'mean_50', 'mean_75', 'mean_mean']
        return self.df
    
    def split_stratified_into_train_val_test(self, random_state=None):
        '''Function to split the data into train,test and validation datasets. 
        :Parameters:
        ------------
            self.df: DataFrame
                DataFrame containing gold and non gold customers
        :Returns:
        ---------
            df_train: DataFrame 
                DataFrame containing training features and target
            df_test:   DataFrame
                DataFrame containing test features and target 
            df_val:    DataFrame
                DataFrame containing validation features and target 
            X_train:   DataFrame
                DataFrame containing training features
            y_train:   DataFrame
                DataFrame containing target variable for training data
            X_val: DataFrame
                DataFrame containing validation features
            y_val: DataFrame
                DataFrame containing target variable for validation data
            X_test: DataFrame
                DataFrame containing test features
            y_test: DataFrame
                DataFrame containing target variable for test data
            df_val_id: DataFrame
                DataFrame containing validation features and target, including id_cols
            list_train: List
                List containing id_cols for training 
        '''
        # getting unique id_col and stratify_col for splitting
        temp_col = id_col + [stratify_col]
        df_target = self.df[temp_col].drop_duplicates()
        X = df_target  # Contains all columns.
        # Dataframe of just the column on which to stratify.
        y = df_target[[stratify_col]]

        # parameter checks
        if train_percent + validation_percent + test_percent != 1.0:
            raise ValueError('fractions %f, %f, %f do not add up to 1.0' %
                            (train_percent, validation_percent, test_percent))

        if stratify_col not in df_target.columns:
            raise ValueError('%s is not a column in the dataframe' %
                            (stratify_col))


        # Split original dataframe into train and temp dataframes.
        self.df_train, df_temp, self.y_train, y_temp = train_test_split(X, y, stratify=y,
                                                                test_size=(1.0 - train_percent),
                                                                random_state=random_state)

        # Split the temp dataframe into val and test dataframes.
        relative_test_percent = test_percent / (validation_percent + test_percent)
        self.df_val, self.df_test, self.y_val, self.y_test = train_test_split(df_temp, y_temp,
                                                            stratify=y_temp,
                                                            test_size=relative_test_percent,
                                                            random_state=random_state)

        assert len(df_target) == len(self.df_train) + len(self.df_val) + len(self.df_test)

        # getting list of id_cols of each df
        list_train = np.array(self.df_train[id_col])
        list_train = [tuple(i) for i in list_train]
        list_test = np.array(self.df_test[id_col])
        list_test = [tuple(i) for i in list_test]
        list_val = np.array(self.df_val[id_col])
        list_val = [tuple(i) for i in list_val]

        # creating train/test/val data, removing identifying columns as they are not features. 
        # identifying columns include id_col and position_col
        temp_col = id_col + position_col
        self.df_train = self.df[self.df[id_col].apply(tuple, axis = 1).isin(list_train)]
        self.df_train.drop(columns=temp_col, inplace=True)
        self.df_test = self.df[self.df[id_col].apply(tuple, axis = 1).isin(list_test)]
        self.df_test.drop(columns=temp_col, inplace=True)
        self.df_val = self.df[self.df[id_col].apply(tuple, axis = 1).isin(list_val)]
        df_val_id = self.df_val.copy()
        self.df_val.drop(columns=temp_col, inplace=True)


        # printing percentages
        print("train target percentage:", len(self.df_train[self.df_train[target_col] == '1'])/len(self.df_train))
        print("test target percentage:", len(self.df_test[self.df_test[target_col] == '1'])/len(self.df_test))
        print("val target percentage:", len(self.df_val[self.df_val[target_col] == '1'])/len(self.df_val))

        # printing df shape
        print("train data shape:", self.df_train.shape)
        print("validation data shape:", self.df_val.shape)
        print("test data shape:", self.df_test.shape)

        # separating df from target column: features -> X | target -> y
        self.X_train = self.df_train.drop(columns=target_col).reset_index(drop=True)
        self.y_train = pd.DataFrame(self.df_train[target_col]).reset_index(drop=True)
        self.X_val = self.df_val.drop(columns=target_col).reset_index(drop=True)
        self.y_val = pd.DataFrame(self.df_val[target_col]).reset_index(drop=True)
        self.X_test = self.df_test.drop(columns=target_col).reset_index(drop=True)
        self.y_test = pd.DataFrame(self.df_test[target_col]).reset_index(drop=True)

        return(self.df_train, self.df_test, self.df_val, self.X_train, self.y_train,
                self.X_val, self.y_val, self.X_test, self.y_test, df_val_id, list_train)


    def encoding_train(self):
        '''Function to encode categorical features for training data.
        :Parameters:
        -----------
        :Returns:
        ---------
            x_df: DataFrame
                DataFrame containing all encoded and standardized columns
            pipe: Pipeline
                Pipeline for encoding and standardization
        '''

        # piping the encoding
        numeric_encoder = Pipeline([('scale', MinMaxScaler())])
        target_encoder = Pipeline([('target_encode', 
                                        TargetEncoder(cols = target_encode_cols, 
                                        handle_unknown='return_nan'))])
        preprocessor = ColumnTransformer(transformers = [
                                            ("num", numeric_encoder, numeric_cols),
                                            ("target_encode", target_encoder, target_encode_cols)], 
                                            remainder = 'passthrough')

        # getting list of column names to map
        self.columns_to_map = numeric_cols + target_encode_cols

        print('columns after preprocessing :', self.columns_to_map,  '\n')

        # applying encoding on columns in df and creating pipeline
        self.X_train = pd.DataFrame({col: vals for vals, col in zip(preprocessor.fit_transform(self.X_train, self.y_train).T, self.columns_to_map)})
        self.pipe = Pipeline(steps=[("preprocessor", preprocessor)])
        self.pipe = self.pipe.fit(self.X_train, self.y_train)

        return self.X_train, self.pipe
    

    def encoding_test_val(self, test = True):
        '''Function to encode categorical features for test and validation data. 
        :Parameters:
        -----------
            test: Boolean
                if test = True, encode for X_test. if False, encode for X_val
        :Returns:
        ---------
            x_output: DataFrame
                DataFrame containing all encoded and standardized columns
        '''

        # applying encoding on columns in df
            # for test
        if test == True:
            self.X_test = pd.DataFrame({col:vals for vals,col in zip(self.pipe.transform(self.X_test).T, self.columns_to_map)})
            return self.X_test
            # for validation
        else:
            self.X_val = pd.DataFrame({col:vals for vals,col in zip(self.pipe.transform(self.X_val).T, self.columns_to_map)})
            return self.X_val
        
        

    def oversample_undersample(x_df, y_df, sampling=True):
        '''Function to oversample minority and undersample majority.
        :Parameters:
        ------------
            x_df: DataFrame
                DataFrame containing training data only for first purchase date excluding the target variable 
            y_df: DataFrame
                DataFrame containing target variable for training data only for first purchase date
        :Returns:
        ---------
            x_df: DataFrame
                DataFrame contiaining oversampled and understampled training data only for first purchase date excluding target variable
            y_df: DataFrame
                DataFrame containing oversampled and undersampled target variable for training data only for first purchase date
        '''

        if sampling:
            undersample = RandomUnderSampler(sampling_strategy = undersampling_strategy, random_state=seed)
            oversample = SMOTE(sampling_strategy = oversampling_strategy, random_state=seed)
            x_df, y_df = undersample.fit_resample(x_df, y_df)
            x_df, y_df = oversample.fit_resample(x_df, y_df)
        return x_df, y_df

In [6]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
data_df = pd.read_parquet("data.parquet")

In [8]:
json_data_path = "data.json"
info_data_path = "data.info"
data = [json.loads(line) for line in open(json_data_path, 'r')]
with open(info_data_path, 'r') as f:
    info = f.read().splitlines()

In [9]:
info_list = [info[i].split(",") for i in range(len(info))]
info_df = pd.DataFrame(info_list[1:]) 
info_df.columns = info_list[0]
info_df.to_parquet("info.parquet")

In [101]:
complete_df = data_df.merge(info_df, how = "left", left_on = ["transcript", "position"], 
                            right_on = ["transcript_id", "transcript_position"])

In [118]:
new_df = complete_df.groupby(["transcript","position","nucleotides","gene_id","transcript_id","transcript_position","label"],as_index=False).quantile(0.5)

In [119]:
new_df = new_df.drop(columns = ['transcript_id','transcript_position'])

In [120]:
import numpy as np
def onehote(sequence):
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    seq2 = [mapping[i] for i in sequence]
    return np.eye(4)[seq2]

In [122]:
new_df

,transcript,position,nucleotides,gene_id,label,dwelling_time,std,mean
0,ENST00000000233,244,AGACC,ENSG00000004059,0,0.007970,6.650,126.0
1,ENST00000000233,261,AAACT,ENSG00000004059,0,0.005885,3.000,108.0
2,ENST00000000233,316,AAACA,ENSG00000004059,0,0.006310,3.780,99.5
3,ENST00000000233,332,GAACA,ENSG00000004059,0,0.007320,2.635,97.5
4,ENST00000000233,368,GGACA,ENSG00000004059,0,0.010500,5.660,122.0
...,...,...,...,...,...,...,...,...
121833,ENST00000641834,610,GAACT,ENSG00000167747,1,0.004570,3.440,101.0
121834,ENST00000641834,697,TGACC,ENSG00000167747,0,0.006440,6.920,122.0
121835,ENST00000641834,864,GGACA,ENSG00000167747,1,0.009160,6.050,111.0
121836,ENST00000641834,926,GGACA,ENSG00000167747,1,0.009085,5.600,115.5


In [121]:
''.join(new_df.nucleotides)

'...'

In [123]:
a = onehote(''.join(new_df.nucleotides))

In [126]:
len(a)/5

121838.0

In [128]:
start = 0
end = 5
for i in range(len(new_df)):
    new_df.nucleotides[i] = a[start:end]
    start = end
    end += 5

<ipython-input-128-f9848faa287a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.nucleotides[i] = a[start:end]


In [129]:
new_df.head()

,transcript,position,nucleotides,gene_id,label,dwelling_time,std,mean
0,ENST00000000233,244,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000004059,0,0.007970,6.650,126.0
1,ENST00000000233,261,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0,0.005885,3.000,108.0
2,ENST00000000233,316,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0,0.006310,3.780,99.5
3,ENST00000000233,332,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0,0.007320,2.635,97.5
4,ENST00000000233,368,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000004059,0,0.010500,5.660,122.0


In [130]:
new_df

,transcript,position,nucleotides,gene_id,label,dwelling_time,std,mean
0,ENST00000000233,244,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000004059,0,0.007970,6.650,126.0
1,ENST00000000233,261,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0,0.005885,3.000,108.0
2,ENST00000000233,316,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0,0.006310,3.780,99.5
3,ENST00000000233,332,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0,0.007320,2.635,97.5
4,ENST00000000233,368,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000004059,0,0.010500,5.660,122.0
...,...,...,...,...,...,...,...,...
121833,ENST00000641834,610,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000167747,1,0.004570,3.440,101.0
121834,ENST00000641834,697,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000167747,0,0.006440,6.920,122.0
121835,ENST00000641834,864,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000167747,1,0.009160,6.050,111.0
121836,ENST00000641834,926,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000167747,1,0.009085,5.600,115.5


In [134]:
df_pre = Preprocessing(new_df)

In [135]:
df_pre.model_features_and_clean()

,transcript,position,nucleotides,gene_id,dwelling_time,std,mean,label
0,ENST00000000233,244,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000004059,0.007970,6.650,126.0,0
1,ENST00000000233,261,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0.005885,3.000,108.0,0
2,ENST00000000233,316,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0.006310,3.780,99.5,0
3,ENST00000000233,332,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000004059,0.007320,2.635,97.5,0
4,ENST00000000233,368,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000004059,0.010500,5.660,122.0,0
...,...,...,...,...,...,...,...,...
121833,ENST00000641834,610,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",ENSG00000167747,0.004570,3.440,101.0,1
121834,ENST00000641834,697,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000167747,0.006440,6.920,122.0,0
121835,ENST00000641834,864,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000167747,0.009160,6.050,111.0,1
121836,ENST00000641834,926,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",ENSG00000167747,0.009085,5.600,115.5,1


In [136]:
df_pre.feature_eng()

TypeError: unhashable type: 'numpy.ndarray'

In [137]:
df_train, df_test, df_val, X_train, y_train, X_val, y_val, X_test, y_test, df_val_id, list_train = df_pre.split_stratified_into_train_val_test(random_state=42)

/Users/xinrantao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


train target percentage: 0.04947758615860092
test target percentage: 0.06195483331341857
val target percentage: 0.06371401825415399
train data shape: (107099, 5)
validation data shape: (17092, 5)
test data shape: (16738, 5)


In [138]:
X_train

,nucleotides,dwelling_time,std,mean
0,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",0.007970,6.650,126.0
1,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",0.005885,3.000,108.0
2,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",0.006310,3.780,99.5
3,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",0.007320,2.635,97.5
4,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",0.010500,5.660,122.0
...,...,...,...,...
107094,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",0.004570,3.440,101.0
107095,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...",0.006440,6.920,122.0
107096,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",0.009160,6.050,111.0
107097,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...",0.009085,5.600,115.5
